In [ ]:

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import PorterStemmer
from nltk.corpus import stopwords
#from nltk.stemporter import PorterStemmer

# Import libraries for feature 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")


from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /gdrive
/gdrive


In [ ]:
#Read files
textfile = r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 4/Comments.csv'
textData = pd.read_csv(textfile) #creates a dataframe

CustInfofile = r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 4/Customers.csv'
CustInfoData = pd.read_csv(CustInfofile)  #creates a dataframe

print(textData.shape)
print(CustInfoData.shape)


(2070, 2)
(2070, 17)


In [ ]:
#Extract target column from Customer Info file
y = CustInfoData["TARGET"]
X_train = CustInfoData.drop(columns=["TARGET"]) #extracting training data without the target column
                     
print(X_train.shape)
print(textData.shape)
print(textData.head())
print(y)

(2070, 16)
(2070, 2)
     ID                                           Comments
0  1309  Does not like the way the phone works. It is t...
1  3556  Wanted to know the nearest store location. Wan...
2  2230  Wants to know how to do text messaging. Referr...
3  2312  Asked how to disable call waiting. referred hi...
4  3327  Needs help learning how to use the phone. I su...
0       Cancelled
1         Current
2         Current
3         Current
4       Cancelled
          ...    
2065    Cancelled
2066    Cancelled
2067    Cancelled
2068    Cancelled
2069    Cancelled
Name: TARGET, Length: 2070, dtype: object


In [ ]:
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

export_csv = textData.to_csv(r'/gdrive/My Drive/ASU_MSBA/CIS_508/5A/TextDataTokenized1.csv')

print(textData.head())


     ID  ...                                  CommentsTokenized
0  1309  ...  [Does, not, like, the, way, the, phone, works,...
1  3556  ...  [Wanted, to, know, the, nearest, store, locati...
2  2230  ...  [Wants, to, know, how, to, do, text, messaging...
3  2312  ...  [Asked, how, to, disable, call, waiting, ., re...
4  3327  ...  [Needs, help, learning, how, to, use, the, pho...

[5 rows x 3 columns]


In [ ]:
# Use English stemmer.
stemmer2 = PorterStemmer()
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)

#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer2.stem(y) for y in x]) # Stem every word.

export_csv = newTextData.to_csv(r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 4/newTextDataPS.csv')

In [ ]:
# Use English stemmer.
stemmer1 = SnowballStemmer("english")
#Tokenize - Split the sentences to lists of words
textData['CommentsTokenized'] = textData['Comments'].apply(word_tokenize)


#Now do stemming - create a new dataframe to store stemmed version
newTextData=pd.DataFrame()
newTextData=textData.drop(columns=["CommentsTokenized","Comments"])
newTextData['CommentsTokenizedStemmed'] = textData['CommentsTokenized'].apply(lambda x: [stemmer1.stem(y) for y in x]) # Stem every word.

export_csv = newTextData.to_csv(r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 4/newTextDataSNB.csv')


In [ ]:
#Join stemmed strings
newTextData['CommentsTokenizedStemmed'] = newTextData['CommentsTokenizedStemmed'].apply(lambda x: " ".join(x))

In [ ]:
#Do Bag-Of-Words model - Term - Document Matrix
#Learn the vocabulary dictionary and return term-document matrix.
#count_vect = CountVectorizer(stop_words=None)
count_vect = CountVectorizer(stop_words='english',lowercase=False)
TD_counts = count_vect.fit_transform(newTextData.CommentsTokenizedStemmed)
print(TD_counts.shape)
print(TD_counts.dtype)
print(count_vect.get_feature_names())
#print(TD_counts)
DF_TD_Counts=pd.DataFrame(TD_counts.toarray())
print(DF_TD_Counts)
export_csv = DF_TD_Counts.to_csv(r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 4//TD_counts-TokenizedStemmed.csv')

(2070, 354)
int64
['3399', '3g', 'abysm', 'access', 'accessori', 'adapt', 'add', 'addit', 'additon', 'address', 'adit', 'adress', 'advertis', 'afraid', 'alway', 'angel', 'angri', 'ani', 'anoth', 'anyth', 'anytim', 'area', 'asap', 'ask', 'bad', 'basic', 'bateri', 'batteri', 'becaus', 'believ', 'better', 'bigger', 'book', 'bought', 'brain', 'bring', 'built', 'busi', 'button', 'buy', 'cancel', 'cancer', 'car', 'care', 'carrier', 'caus', 'cc', 'cell', 'certain', 'chang', 'charg', 'charger', 'check', 'chip', 'citi', 'claim', 'cleariti', 'cold', 'comapr', 'compani', 'compar', 'competit', 'complain', 'complaint', 'concept', 'connect', 'consisit', 'consist', 'constan', 'contact', 'continu', 'contract', 'correct', 'cost', 'coupl', 'cover', 'coverag', 'creat', 'credit', 'cstmer', 'cstmr', 'current', 'cust', 'custom', 'customr', 'date', 'day', 'dead', 'decent', 'defect', 'deo', 'did', 'die', 'differ', 'difficult', 'digiti', 'direct', 'disabl', 'doe', 'don', 'dont', 'drop', 'dure', 'easier', 'effe

In [ ]:
#Compute TF-IDF Matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(TD_counts)
print(X_train_tfidf.shape)
DF_TF_IDF=pd.DataFrame(X_train_tfidf.toarray())
print(DF_TF_IDF)
export_csv= DF_TF_IDF.to_csv(r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 4/TFIDF_counts-TokenizedStemmed.csv')

(2070, 354)
      0    1    2    3        4    5    ...  348  349  350  351  352  353
0     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.27568  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4     0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
...   ...  ...  ...  ...      ...  ...  ...  ...  ...  ...  ...  ...  ...
2065  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2066  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2067  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2068  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2069  0.0  0.0  0.0  0.0  0.00000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

[2070 rows x 354 columns]


In [ ]:
id= textData.iloc[:,0]
combined=pd.concat([id,DF_TF_IDF], axis=1)
combined1=X_train.merge(combined, on=['ID'])
DF_TF_IDF=combined1.drop(columns=['ID'])

In [ ]:
# One Hot Encoding
#The below function returns a list of categorical features which are not numeric. 
cat_cols = DF_TF_IDF.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(cat_cols.shape)
print(cat_cols)

# OneHotEncoding is to be done on Categorical variables.
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(DF_TF_IDF[cat_cols]),columns=ohe.get_feature_names(),index=DF_TF_IDF.index)
DF_TF_IDF = pd.concat([DF_TF_IDF,Xcat],axis=1)
DF_TF_IDF.drop(labels=cat_cols,axis=1,inplace=True)
DF_TF_IDF.sample(5)



(6,)
Index(['Sex', 'Status', 'Car_Owner', 'Paymethod', 'LocalBilltype',
       'LongDistanceBilltype'],
      dtype='object')


,Children,Est_Income,Usage,Age,RatePlan,LongDistance,International,Local,Dropped,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,...,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,x0_F,x0_M,x1_D,x1_M,x1_S,x2_N,x2_Y,x3_Auto,x3_CC,x3_CH,x4_Budget,x4_FreeLocal,x5_Intnl_discount,x5_Standard
522,0,96681.0,73.76,19.760000,2,2.00,7.66,40.92,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.772949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1234,2,64816.6,27.87,31.126667,4,21.00,0.00,6.87,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.204572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1062,2,36725.1,94.88,55.793333,4,8.65,9.22,77.00,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.293881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1244,0,64319.2,40.16,41.226667,1,3.01,0.00,37.15,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1679,0,95405.7,13.96,46.506667,4,7.74,0.00,6.21,0,0.0,0.0,0.0,0.0,0.368149,0.0,0.0,0.334347,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.416738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
#Feature selection Filter Method Chi square
new_DF_TF_IDF = SelectKBest(score_func=chi2, k=50).fit_transform(DF_TF_IDF,y)
print(new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)

#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_SelectedFeatures,y)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_SelectedFeatures,y)))
rf_predictions = clf.predict(DF_TF_IDF_SelectedFeatures)
print("Confusion Matrix:")
print(confusion_matrix(y, rf_predictions))
print("Classification Report")
print(classification_report(y, rf_predictions))

#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')
DF_TF_IDF_Filter50=DF_TF_IDF_SelectedFeatures

(2070, 50)
       0         1       2      3     4   ...   45   46   47   48   49
0     1.0  38000.00  229.64  23.56  0.00  ...  1.0  0.0  0.0  1.0  0.0
1     2.0  29616.00   75.29  29.78  0.00  ...  0.0  1.0  1.0  0.0  0.0
2     0.0  19732.80   47.25  24.81  0.00  ...  0.0  1.0  1.0  0.0  0.0
3     2.0     96.33   59.01  26.13  0.00  ...  0.0  1.0  0.0  1.0  0.0
4     2.0  52004.80   28.14   5.03  0.00  ...  1.0  0.0  1.0  0.0  0.0
...   ...       ...     ...    ...   ...  ...  ...  ...  ...  ...  ...
2065  0.0  78851.30   29.04   0.37  0.00  ...  1.0  0.0  0.0  1.0  0.0
2066  1.0  17540.70   36.20  22.17  0.57  ...  1.0  0.0  0.0  1.0  1.0
2067  0.0  83891.90   74.40  28.92  0.00  ...  1.0  0.0  1.0  0.0  0.0
2068  2.0  28220.80   38.95  26.49  0.00  ...  1.0  0.0  1.0  0.0  0.0
2069  0.0  28589.10  100.28  13.19  0.00  ...  1.0  0.0  0.0  1.0  0.0

[2070 rows x 50 columns]
Accuracy score (training): 0.926570
Confusion Matrix:
[[ 729   75]
 [  77 1189]]
Classification Report
        

In [ ]:
#Feature selection Filter Method Chi square
new_DF_TF_IDF = SelectKBest(score_func=chi2, k=10).fit_transform(DF_TF_IDF,y)
print(new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)


#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_SelectedFeatures,y)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_SelectedFeatures,y)))
rf_predictions = clf.predict(DF_TF_IDF_SelectedFeatures)
print("Confusion Matrix:")
print(confusion_matrix(y, rf_predictions))
print("Classification Report")
print(classification_report(y, rf_predictions))

#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')
DF_TF_IDF_Filter10=DF_TF_IDF_SelectedFeatures

(2070, 10)
        0         1       2      3     4       5    6    7    8    9
0     1.0  38000.00  229.64  23.56  0.00  206.08  0.0  0.0  0.0  1.0
1     2.0  29616.00   75.29  29.78  0.00   45.50  0.0  0.0  1.0  0.0
2     0.0  19732.80   47.25  24.81  0.00   22.44  0.0  0.0  1.0  0.0
3     2.0     96.33   59.01  26.13  0.00   32.88  0.0  0.0  0.0  1.0
4     2.0  52004.80   28.14   5.03  0.00   23.11  0.0  0.0  1.0  0.0
...   ...       ...     ...    ...   ...     ...  ...  ...  ...  ...
2065  0.0  78851.30   29.04   0.37  0.00   28.66  0.0  0.0  0.0  1.0
2066  1.0  17540.70   36.20  22.17  0.57   13.45  0.0  0.0  0.0  1.0
2067  0.0  83891.90   74.40  28.92  0.00   45.47  0.0  0.0  1.0  0.0
2068  2.0  28220.80   38.95  26.49  0.00   12.46  0.0  0.0  1.0  0.0
2069  0.0  28589.10  100.28  13.19  0.00   87.09  0.0  0.0  0.0  1.0

[2070 rows x 10 columns]
Accuracy score (training): 0.917874
Confusion Matrix:
[[ 719   85]
 [  85 1181]]
Classification Report
              precision    recal

In [ ]:
#Feature selection Filter Method
new_DF_TF_IDF = SelectKBest(score_func=chi2, k=15).fit_transform(DF_TF_IDF,y)
print(new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)


#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_SelectedFeatures,y)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_SelectedFeatures,y)))
rf_predictions = clf.predict(DF_TF_IDF_SelectedFeatures)
print("Confusion Matrix:")
print(confusion_matrix(y, rf_predictions))
print("Classification Report")
print(classification_report(y, rf_predictions))

#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')
DF_TF_IDF_Filter15=DF_TF_IDF_SelectedFeatures

(2070, 15)
       0         1       2      3     4   ...       10   11   12   13   14
0     1.0  38000.00  229.64  23.56  0.00  ...  0.00000  0.0  0.0  0.0  1.0
1     2.0  29616.00   75.29  29.78  0.00  ...  0.37653  0.0  1.0  1.0  0.0
2     0.0  19732.80   47.25  24.81  0.00  ...  0.37653  0.0  1.0  1.0  0.0
3     2.0     96.33   59.01  26.13  0.00  ...  0.37653  0.0  1.0  0.0  1.0
4     2.0  52004.80   28.14   5.03  0.00  ...  0.37653  0.0  0.0  1.0  0.0
...   ...       ...     ...    ...   ...  ...      ...  ...  ...  ...  ...
2065  0.0  78851.30   29.04   0.37  0.00  ...  0.00000  0.0  0.0  0.0  1.0
2066  1.0  17540.70   36.20  22.17  0.57  ...  0.00000  0.0  0.0  0.0  1.0
2067  0.0  83891.90   74.40  28.92  0.00  ...  0.00000  0.0  0.0  1.0  0.0
2068  2.0  28220.80   38.95  26.49  0.00  ...  0.00000  0.0  0.0  1.0  0.0
2069  0.0  28589.10  100.28  13.19  0.00  ...  0.00000  0.0  0.0  0.0  1.0

[2070 rows x 15 columns]
Accuracy score (training): 0.923188
Confusion Matrix:
[[ 730   

In [ ]:
# Feature selection Filter Method Chi square and k=50
# Mean Accuracy Score - ON Text:  0.879707311024003
#Feature selection Filter Method Chi square and k=10
#Mean Accuracy Score - ON Text:  0.862332559039876
#Feature selection Filter Method Chi square and k=15
#Mean Accuracy Score - ON Text:  0.8737007295296169

In [ ]:
# Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(DF_TF_IDF_Filter50, y, test_size = 0.2, random_state = 1)

#Construct a Random Forest Classifier on combined data
clf=RandomForestClassifier()
RF_Comb = clf.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_train, y_train)))
rf_predictions = clf.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_predictions))
print("Classification Report")
print(classification_report(y_test, rf_predictions))

#run cross-validation - COMBINED Data
rf_Comb_cv_score = cross_val_score(RF_Comb, DF_TF_IDF_Filter50, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_Comb_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",rf_Comb_cv_score.mean())
print('\n')


Accuracy score (training): 0.930556
Confusion Matrix:
[[128  22]
 [ 30 234]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.81      0.85      0.83       150
     Current       0.91      0.89      0.90       264

    accuracy                           0.87       414
   macro avg       0.86      0.87      0.87       414
weighted avg       0.88      0.87      0.88       414

=== All Accuracy Scores ===
[0.82791328 0.86391792 0.90301974 0.89566396 0.825      0.9015625
 0.8265625  0.809375   0.8296875  0.89375    0.88075397 0.89325397
 0.93412698 0.76507937 0.90575397 0.86031746 0.90238095 0.93869048
 0.89444444 0.91825397]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8734753980352303




In [ ]:
#Feature selection Filter Method ANOVA
new_DF_TF_IDF = SelectKBest(score_func=f_classif, k=50).fit_transform(DF_TF_IDF,y)
print(new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)

#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_SelectedFeatures,y)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_SelectedFeatures,y)))
rf_predictions = clf.predict(DF_TF_IDF_SelectedFeatures)
print("Confusion Matrix:")
print(confusion_matrix(y, rf_predictions))
print("Classification Report")
print(classification_report(y, rf_predictions))

#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')
DF_TF_IDF_Filter50=DF_TF_IDF_SelectedFeatures

(2070, 50)
       0         1       2      3     4   ...   45   46   47   48   49
0     1.0  38000.00  229.64  23.56  0.00  ...  1.0  0.0  0.0  1.0  0.0
1     2.0  29616.00   75.29  29.78  0.00  ...  0.0  1.0  1.0  0.0  0.0
2     0.0  19732.80   47.25  24.81  0.00  ...  0.0  1.0  1.0  0.0  0.0
3     2.0     96.33   59.01  26.13  0.00  ...  0.0  1.0  0.0  1.0  0.0
4     2.0  52004.80   28.14   5.03  0.00  ...  1.0  0.0  1.0  0.0  0.0
...   ...       ...     ...    ...   ...  ...  ...  ...  ...  ...  ...
2065  0.0  78851.30   29.04   0.37  0.00  ...  1.0  0.0  0.0  1.0  0.0
2066  1.0  17540.70   36.20  22.17  0.57  ...  1.0  0.0  0.0  1.0  1.0
2067  0.0  83891.90   74.40  28.92  0.00  ...  1.0  0.0  1.0  0.0  0.0
2068  2.0  28220.80   38.95  26.49  0.00  ...  1.0  0.0  1.0  0.0  0.0
2069  0.0  28589.10  100.28  13.19  0.00  ...  1.0  0.0  0.0  1.0  0.0

[2070 rows x 50 columns]
Accuracy score (training): 0.926570
Confusion Matrix:
[[ 741   63]
 [  89 1177]]
Classification Report
        

In [ ]:
#Feature selection Filter Method ANOVA
new_DF_TF_IDF = SelectKBest(score_func=f_classif, k=10).fit_transform(DF_TF_IDF,y)
print(new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)


#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_SelectedFeatures,y)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_SelectedFeatures,y)))
rf_predictions = clf.predict(DF_TF_IDF_SelectedFeatures)
print("Confusion Matrix:")
print(confusion_matrix(y, rf_predictions))
print("Classification Report")
print(classification_report(y, rf_predictions))

#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')
DF_TF_IDF_Filter10=DF_TF_IDF_SelectedFeatures

(2070, 10)
        0        1        2         3    4        5    6         7    8    9
0     1.0  0.00000  0.00000  0.000000  0.0  0.00000  0.0  0.000000  0.0  1.0
1     2.0  0.37653  0.37653  0.271105  0.0  0.37653  0.0  0.348322  1.0  0.0
2     0.0  0.37653  0.37653  0.271105  0.0  0.37653  0.0  0.348322  1.0  0.0
3     2.0  0.37653  0.37653  0.271105  0.0  0.37653  0.0  0.348322  0.0  1.0
4     2.0  0.37653  0.37653  0.271105  0.0  0.37653  0.0  0.348322  1.0  0.0
...   ...      ...      ...       ...  ...      ...  ...       ...  ...  ...
2065  0.0  0.00000  0.00000  0.000000  0.0  0.00000  0.0  0.000000  0.0  1.0
2066  1.0  0.00000  0.00000  0.000000  0.0  0.00000  0.0  0.000000  0.0  1.0
2067  0.0  0.00000  0.00000  0.000000  0.0  0.00000  0.0  0.000000  1.0  0.0
2068  2.0  0.00000  0.00000  0.000000  0.0  0.00000  0.0  0.000000  1.0  0.0
2069  0.0  0.00000  0.00000  0.000000  0.0  0.00000  0.0  0.000000  0.0  1.0

[2070 rows x 10 columns]
Accuracy score (training): 0.784541
Con

In [ ]:
#Feature selection Filter Method ANOVA
new_DF_TF_IDF = SelectKBest(score_func=f_classif, k=15).fit_transform(DF_TF_IDF,y)
print(new_DF_TF_IDF.shape)

DF_TF_IDF_SelectedFeatures= pd.DataFrame(new_DF_TF_IDF)
print(DF_TF_IDF_SelectedFeatures)


#Construct a Random Forest Classifier on text data
clf=RandomForestClassifier()
RF_text = clf.fit(DF_TF_IDF_SelectedFeatures,y)
print("Accuracy score (training): {0:.6f}".format(clf.score(DF_TF_IDF_SelectedFeatures,y)))
rf_predictions = clf.predict(DF_TF_IDF_SelectedFeatures)
print("Confusion Matrix:")
print(confusion_matrix(y, rf_predictions))
print("Classification Report")
print(classification_report(y, rf_predictions))

#run cross-validation - Text Data
clf_cv_score = cross_val_score(clf, DF_TF_IDF_SelectedFeatures, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",clf_cv_score.mean())
print('\n')
DF_TF_IDF_Filter15=DF_TF_IDF_SelectedFeatures

(2070, 15)
       0    1    2        3    4    5   ...   9        10   11        12   13   14
0     1.0  0.0  0.0  0.00000  0.0  0.0  ...  0.0  0.00000  0.0  0.000000  0.0  1.0
1     2.0  0.0  0.0  0.37653  0.0  0.0  ...  0.0  0.37653  0.0  0.348322  1.0  0.0
2     0.0  0.0  0.0  0.37653  0.0  0.0  ...  0.0  0.37653  0.0  0.348322  1.0  0.0
3     2.0  0.0  0.0  0.37653  0.0  0.0  ...  0.0  0.37653  0.0  0.348322  0.0  1.0
4     2.0  0.0  0.0  0.37653  0.0  0.0  ...  0.0  0.37653  0.0  0.348322  1.0  0.0
...   ...  ...  ...      ...  ...  ...  ...  ...      ...  ...       ...  ...  ...
2065  0.0  0.0  0.0  0.00000  0.0  0.0  ...  0.0  0.00000  0.0  0.000000  0.0  1.0
2066  1.0  0.0  0.0  0.00000  0.0  0.0  ...  0.0  0.00000  0.0  0.000000  0.0  1.0
2067  0.0  0.0  0.0  0.00000  0.0  0.0  ...  0.0  0.00000  0.0  0.000000  1.0  0.0
2068  2.0  0.0  0.0  0.00000  0.0  0.0  ...  0.0  0.00000  0.0  0.000000  1.0  0.0
2069  0.0  0.0  0.0  0.00000  0.0  0.0  ...  0.0  0.00000  0.0  0.000000  0.

In [ ]:
# Feature selection Filter Method ANOVA and k=50
# Mean Accuracy Score - ON Text:  0.8764379476384049
#Feature selection Filter Method ANOVA and k=10
#Mean Accuracy Score - ON Text:  0.7982404483643052
#Feature selection Filter Method ANOVA and k=15
#Mean Accuracy Score - ON Text:  0.7986372737611306

In [ ]:
# Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(DF_TF_IDF_Filter50, y, test_size = 0.2, random_state = 1)

#Construct a Gradient Boosting Classifier on combined data
clf=GradientBoostingClassifier()
RF_Comb = clf.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_train, y_train)))
rf_predictions = clf.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_predictions))
print("Classification Report")
print(classification_report(y_test, rf_predictions))

#run cross-validation - COMBINED Data
rf_Comb_cv_score = cross_val_score(RF_Comb, DF_TF_IDF, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_Comb_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",rf_Comb_cv_score.mean())
print('\n')


Accuracy score (training): 0.903986
Confusion Matrix:
[[129  21]
 [ 32 232]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.80      0.86      0.83       150
     Current       0.92      0.88      0.90       264

    accuracy                           0.87       414
   macro avg       0.86      0.87      0.86       414
weighted avg       0.88      0.87      0.87       414

=== All Accuracy Scores ===
[0.86817654 0.89566396 0.93960511 0.87127371 0.8625     0.91875
 0.8515625  0.809375   0.8875     0.896875   0.94325397 0.88869048
 0.92956349 0.74801587 0.88869048 0.90238095 0.89325397 0.90912698
 0.84444444 0.92281746]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8835759956929928




In [ ]:
DF_Combined1= pd.DataFrame(DF_TF_IDF)

In [ ]:
#Sequential Forward Search using Decision Tree Classifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

clf = DecisionTreeClassifier()

sfs1 = SFS(clf, 
           k_features=7, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=2)

sfs1 = sfs1.fit(DF_Combined1,y)
sfs1.subsets_


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 377 out of 377 | elapsed:    3.5s finished

[2020-10-30 23:25:54] Features: 1/7 -- score: 0.7932367149758455[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 376 out of 376 | elapsed:    4.1s finished

[2020-10-30 23:25:58] Features: 2/7 -- score: 0.8217391304347826[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 375 out of 375 | elapsed:    5.1s finished

[2020-10-30 23:26:03] Features: 3/7 -- score: 0.8415458937198068[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

{1: {'avg_score': 0.7932367149758455,
  'cv_scores': array([0.74202899, 0.84444444]),
  'feature_idx': (3,),
  'feature_names': ('Age',)},
 2: {'avg_score': 0.8217391304347826,
  'cv_scores': array([0.78164251, 0.86183575]),
  'feature_idx': (3, 363),
  'feature_names': ('Age', 'x0_F')},
 3: {'avg_score': 0.8415458937198068,
  'cv_scores': array([0.79516908, 0.88792271]),
  'feature_idx': (3, 229, 363),
  'feature_names': ('Age', 220, 'x0_F')},
 4: {'avg_score': 0.8478260869565217,
  'cv_scores': array([0.80676329, 0.88888889]),
  'feature_idx': (3, 229, 233, 363),
  'feature_names': ('Age', 220, 224, 'x0_F')},
 5: {'avg_score': 0.8555555555555555,
  'cv_scores': array([0.82318841, 0.88792271]),
  'feature_idx': (3, 229, 233, 306, 363),
  'feature_names': ('Age', 220, 224, 297, 'x0_F')},
 6: {'avg_score': 0.8579710144927536,
  'cv_scores': array([0.82705314, 0.88888889]),
  'feature_idx': (3, 163, 229, 233, 306, 363),
  'feature_names': ('Age', 154, 220, 224, 297, 'x0_F')},
 7: {'avg_s

In [ ]:
#Sequential forward search result
print(sfs1.k_feature_names_)
print(sfs1.k_score_)
# Use features from Sequential forward Selection
feat_cols = list(sfs1.k_feature_idx_)
X_SFS=DF_Combined1.iloc[:, feat_cols]

('Age', 147, 154, 220, 224, 297, 'x0_F')
0.8589371980676328


In [ ]:
# Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_SFS, y, test_size = 0.2, random_state = 1)

#Construct a Random Forest Classifier on combined data
clf=RandomForestClassifier()
RF_Comb = clf.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_train, y_train)))
rf_predictions = clf.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_predictions))
print("Classification Report")
print(classification_report(y_test, rf_predictions))

#run cross-validation - COMBINED Data
rf_Comb_cv_score = cross_val_score(RF_Comb, DF_TF_IDF, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_Comb_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",rf_Comb_cv_score.mean())
print('\n')

Accuracy score (training): 0.918478
Confusion Matrix:
[[129  21]
 [ 26 238]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.83      0.86      0.85       150
     Current       0.92      0.90      0.91       264

    accuracy                           0.89       414
   macro avg       0.88      0.88      0.88       414
weighted avg       0.89      0.89      0.89       414

=== All Accuracy Scores ===
[0.84436702 0.87611305 0.89508324 0.89140534 0.8953125  0.921875
 0.8765625  0.821875   0.8796875  0.88125    0.91825397 0.90119048
 0.93075397 0.7968254  0.90575397 0.86488095 0.91031746 0.93075397
 0.90238095 0.91031746]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8877479856271778




In [ ]:
#Sequential Forward Search USing KNN Classifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

clf = KNeighborsClassifier()

sfs1 = SFS(clf, 
           k_features=7, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=2)

sfs1 = sfs1.fit(DF_Combined1,y)
sfs1.subsets_

#Sequential forward search result
print(sfs1.k_feature_names_)
print(sfs1.k_score_)
# Use features from Sequential forward Selection
feat_cols = list(sfs1.k_feature_idx_)
X_SFS=DF_Combined1.iloc[:, feat_cols]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 377 out of 377 | elapsed:   31.1s finished

[2020-10-30 20:22:54] Features: 1/7 -- score: 0.8884057971014493[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 376 out of 376 | elapsed:   19.2s finished

[2020-10-30 20:23:13] Features: 2/7 -- score: 0.8956521739130435[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 375 out of 375 | elapsed:   19.9s finished

[2020-10-30 20:23:33] Features: 3/7 -- score: 0.8966183574879227[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 

('Usage', 22, 27, 92, 110, 'x0_F', 'x0_M')
0.8990338164251208


[Parallel(n_jobs=1)]: Done 371 out of 371 | elapsed:   20.7s finished

[2020-10-30 20:24:55] Features: 7/7 -- score: 0.8990338164251208

In [ ]:
# Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_SFS, y, test_size = 0.2, random_state = 1)

#Construct a Gradient Boosting Classifier on combined data
clf=GradientBoostingClassifier()
RF_Comb = clf.fit(X_train,y_train)
print("Accuracy score (training): {0:.6f}".format(clf.score(X_train, y_train)))
rf_predictions = clf.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, rf_predictions))
print("Classification Report")
print(classification_report(y_test, rf_predictions))

#run cross-validation - COMBINED Data
rf_Comb_cv_score = cross_val_score(RF_Comb, DF_TF_IDF, y, cv=20, scoring="balanced_accuracy")
print("=== All Accuracy Scores ===")
print(rf_Comb_cv_score)
print('\n')
print("=== Mean Accuracy Score ===")
print("Mean Accuracy Score - ON Text: ",rf_Comb_cv_score.mean())
print('\n')

Accuracy score (training): 0.816425
Confusion Matrix:
[[ 94  56]
 [ 31 233]]
Classification Report
              precision    recall  f1-score   support

   Cancelled       0.75      0.63      0.68       150
     Current       0.81      0.88      0.84       264

    accuracy                           0.79       414
   macro avg       0.78      0.75      0.76       414
weighted avg       0.79      0.79      0.79       414

=== All Accuracy Scores ===
[0.86817654 0.89566396 0.93960511 0.87127371 0.8625     0.91875
 0.8515625  0.809375   0.8875     0.896875   0.94325397 0.88869048
 0.92956349 0.74801587 0.88869048 0.90238095 0.89325397 0.90912698
 0.84444444 0.92281746]


=== Mean Accuracy Score ===
Mean Accuracy Score - ON Text:  0.8835759956929928


